In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%cd "/content/drive/My Drive/Data Mining"

/content/drive/My Drive/Data Mining


In [3]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

     |████████████████████████████████| 51kB 2.7MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149804 sha256=d2498cbd641b3f6f3d86e812cc315e00b35efe572ab6602e4dc45bcedde94ff8
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


In [4]:
import pandas as pd
import numpy as np
from pandas import DataFrame

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

from tqdm.notebook import tqdm

import os

In [5]:
# df_data = pd.read_csv("../Dataset/processedDataWithThaliaAdded.csv", nrows=10000,delimiter='|')
df_data = pd.read_csv("Data/processedDataWithThaliaAdded.csv", sep = "|")
df_data.head(5)

,itemID,title,titleFound,title_thalia,author_raw,author_api,author_thalia,language,language_thalia,collection,isbn,publisher_raw,publisher_api,publishedDate,description,main topic,subtopics,thumbnail,maturityRating,pageCount,isEbook
0,5649,... and the Word became a Story,... and the Word became a Story,and the word became a story,The Author,The Author,the author,en,Englisch,2.0,9.783743e+12,Books on Demand,BoD – Books on Demand,2018-04-17,At the beginning was the Word. And from the Wo...,FL,FL FM FN,http://books.google.com/books/content?id=FrElD...,NOT_MATURE,304.0,False
1,77497,... EVVAI!,... EVVAI!,evvai,Cristina Polacchini,CRISTINA POLACCHINI,cristina polacchini,it,Italienisch,2.0,9.781326e+12,Lulu.com,Lulu.com,2015-08-01,"Amicizia, scuola, amore, alti e bassi, in and ...",YFB,,http://books.google.com/books/content?id=lGNgC...,NOT_MATURE,106.0,False
2,13238,'... then he ate my boy entrancers.',Then He Ate My Boy Entrancers,then he ate my boy entrancers,Louise Rennison,NaN,louise rennison,un,Englisch,2.0,NaN,HarperCollins Publishers,NaN,2008,NaN,YFM,4Z GB ACN YFQ,NaN,NOT_MATURE,NaN,False
3,24800,... trägt Jeans und Tennisschuhe,"""... trägt Jeans und Tennisschuhe""",traegt jeans und tennisschuhe,Frauke Kühn,Frauke Kühn,frauke kuehn,un,Deutsch,2.0,NaN,Rowohlt Repertoire,NaN,1995,NaN,YFCF,5AN 1DFG 1QBDN YFB YFY YXH YXJ 3MPQX ...,NaN,NOT_MATURE,7.0,False
4,26966,TRIPION minaccia dallo spazio,""" TRIPION "" minaccia dallo spazio",tripion minaccia dallo spazio,Stefano Grimaldi,Stefano Grimaldi,stefano grimaldi,it,Italienisch,2.0,9.781292e+12,Lulu.com,Lulu.com,2014-07-06,IL GENERE UMANO E' DESTINATO A SCOMPARIRE...QU...,FL,,http://books.google.com/books/content?id=HuPeB...,NOT_MATURE,58.0,False


In [6]:
df_dataset = df_data[['itemID','title_thalia','author_thalia','publisher_api','main topic','subtopics']]
df_dataset.fillna("",inplace=True)

df_dataset['metadata'] = df_dataset[['main topic','subtopics']].apply(lambda x: ' '.join(x), axis = 1)
#df_dataset['metadata2'] = df_dataset[['author_thalia','publisher_api']].apply(lambda x: ' '.join(x), axis = 1)


df_dataset['metadata'] = df_dataset['metadata'].str.replace('|',' ')
df_dataset.head(5)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

,itemID,title_thalia,author_thalia,publisher_api,main topic,subtopics,metadata
0,5649,and the word became a story,the author,BoD – Books on Demand,FL,FL FM FN,FL FL FM FN
1,77497,evvai,cristina polacchini,Lulu.com,YFB,,YFB
2,13238,then he ate my boy entrancers,louise rennison,,YFM,4Z GB ACN YFQ,YFM 4Z GB ACN YFQ
3,24800,traegt jeans und tennisschuhe,frauke kuehn,,YFCF,5AN 1DFG 1QBDN YFB YFY YXH YXJ 3MPQX ...,YFCF 5AN 1DFG 1QBDN YFB YFY YXH YXJ 3MPQX ...
4,26966,tripion minaccia dallo spazio,stefano grimaldi,Lulu.com,FL,,FL


In [7]:
indices =pd.Series(df_dataset.index,index = df_dataset["title_thalia"]).drop_duplicates()
indices

title_thalia
 and the word became a story                       0
 evvai                                             1
 then he ate my boy entrancers                     2
 traegt jeans und tennisschuhe                     3
 tripion  minaccia dallo spazio                    4
                                               ...  
the wonderful adventures of nils               57454
too close for comfort                          57455
trail of lightning                             57456
virals                                         57457
warriors dawn of the clans 01 the sun trail    57458
Length: 57459, dtype: int64

In [8]:
#Recommendations based on Metadata (Topic/Subtopic)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_dataset['metadata'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), index=df_dataset.index.tolist())

#count2=CountVectorizer(stop_words="english")
#count_matrix2= count.fit_transform(df_dataset['metadata'])

#count_matrix

In [9]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel
from sklearn.kernel_approximation import Nystroem

# Compute the cosine similarity matrix based on the tfidf_matrix
# content_correlation = cosine_similarity(tfidf_matrix, tfidf_matrix)
kernel = Nystroem(random_state=43, gamma=0.2, n_components=5000)
kernel.fit(tfidf_matrix)
content_correlation = kernel.transform(tfidf_matrix)

In [10]:
content_correlation

array([[0.00947394, 0.03199938, 0.0098013 , ..., 0.00969433, 0.03199937,
        0.00894948],
       [0.0097282 , 0.00942388, 0.01941314, ..., 0.00955259, 0.0094239 ,
        0.00874109],
       [0.0107415 , 0.00995118, 0.01057196, ..., 0.01030845, 0.00995119,
        0.01474542],
       ...,
       [0.01043435, 0.00853402, 0.00988408, ..., 0.01075751, 0.00853391,
        0.00951736],
       [0.00895301, 0.00946868, 0.01221704, ..., 0.00867036, 0.0094687 ,
        0.00711525],
       [0.01490086, 0.00923146, 0.00728137, ..., 0.00818149, 0.00923148,
        0.03174716]])

In [47]:
def get_recommendations(title, cosine_sim, column):
    # Get the index of the movie that matches the title
    idx = indices[title]
    # Get the pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores of the top 5 most similar movies
    #sim_scores = sim_scores[1:6]

    # Get relevant books
    books = [i for i in sim_scores if i[0] != idx][0:5]
    # Get the movie indices
    book_indices = [i[0] for i in books]

    # Get the pred scores
    book_pred_score = [i[1] for i in books]

    # Return the top 5 most similar movies
    return pd.DataFrame({'itemID': book_indices,'title_thalia': list(df_dataset['title_thalia'].iloc[book_indices].values), 'metadata': list(df_dataset[column].iloc[book_indices].values), 'Predicted Rating': book_pred_score})

In [19]:
df_dataset.title_thalia[1]

' evvai'

In [31]:
book_search = ' evvai'
df_dataset[df_dataset['title_thalia']==book_search]

,itemID,title_thalia,author_thalia,publisher_api,main topic,subtopics,metadata
1,77497,evvai,cristina polacchini,Lulu.com,YFB,,YFB


In [48]:
book_search = 'luuurve is a many trousered thing'
display(df_dataset[df_dataset.title_thalia==book_search])
get_recommendations('luuurve is a many trousered thing', content_correlation, 'metadata')

,itemID,title_thalia,author_thalia,publisher_api,main topic,subtopics,metadata
27586,18472,luuurve is a many trousered thing,louise rennison,,YFM,5AM,YFM 5AM


,itemID,title_thalia,metadata,Predicted Rating
0,2785,angels and fairies a coloring book,YFJ,0.263124
1,1251,a turminha das cores,YFB,0.263124
2,3947,ballerina dreams,YNDB YFY YNM YXN,0.143436
3,2530,amo mangiare frutta e verdura i love to eat fr...,YBLJ YFP YFP ...,0.137328
4,4739,beyond valor,FMW,0.134222


In [42]:
# Recommendations Based of Author
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix2 = tfidf.fit_transform(df_dataset['author_thalia'])
tfidf_df2 = pd.DataFrame(tfidf_matrix.toarray(), index=df_dataset.index.tolist())

#count2=CountVectorizer(stop_words="english")
#count_matrix2= count.fit_transform(df_dataset['metadata'])

#count_matrix

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix based on the tfidf_matrix
content_correlation2 = linear_kernel(tfidf_matrix2, tfidf_matrix2)

In [ ]:
#Recommendations based on Author
def get_recommendations_author(title, cosine_sim, method_name):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the top 5 most similar movies
    sim_scores = sim_scores[1:6]

    # Get the movie indices
    book_indices = [i[0] for i in sim_scores]

    # Get the pred scores
    book_pred_score = [i[1] for i in sim_scores]

    return pd.DataFrame({'itemID': book_indices,'title_thalia': list(df_dataset['title_thalia'].iloc[book_indices].values), 'author_thalia': list(df_dataset['author_thalia'].iloc[book_indices].values), method_name: book_pred_score})

In [ ]:
book_search = 'famous'
display(df_dataset[df_dataset.title_thalia==book_search])
get_recommendations_author(book_search,content_correlation2, 'Predicted Rating')

,itemID,title_thalia,author_thalia,publisher_api,main topic,subtopics,metadata
1,32836,famous,"jilly gagnon, todd strasser",Planet!,YFM,YXQ,YFM YXQ


,itemID,title_thalia,author_thalia,Predicted Rating
0,5386,blood on my hands,todd strasser,0.663472
1,6420,cant get there from here,todd strasser,0.663472
2,9953,date with a rockstar,sarah gagnon,0.432974
3,4668,belle and moozie and whatshisname,gayle strasser,0.349040
4,8281,cuenta la historia de la quiropráctica,todd waters,0.307443


In [ ]:
# Recommendations Based of Publisher
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix3 = tfidf.fit_transform(df_dataset['publisher_api'])
tfidf_df3 = pd.DataFrame(tfidf_matrix.toarray(), index=df_dataset.index.tolist())

#count2=CountVectorizer(stop_words="english")
#count_matrix2= count.fit_transform(df_dataset['metadata'])

#count_matrix

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix based on the tfidf_matrix
content_correlation3 = linear_kernel(tfidf_matrix3, tfidf_matrix3)

In [ ]:
def get_recommendations_publisher(title, cosine_sim, method_name):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the top 5 most similar movies
    sim_scores = sim_scores[1:6]

    # Get the movie indices
    book_indices = [i[0] for i in sim_scores]

    # Get the pred scores
    book_pred_score = [i[1] for i in sim_scores]

    return pd.DataFrame({'itemID': book_indices,'title_thalia': list(df_dataset['title_thalia'].iloc[book_indices].values), 'publisher_api': list(df_dataset['publisher_api'].iloc[book_indices].values), method_name: book_pred_score})

In [ ]:
book_search = 'famous'
display(df_dataset[df_dataset.title_thalia==book_search])
get_recommendations_publisher(book_search,content_correlation3, 'Predicted Rating')

,itemID,title_thalia,author_thalia,publisher_api,main topic,subtopics,metadata
16238,32836,famous,"jilly gagnon, todd strasser",Planet!,YFM,YXQ,YFM YXQ


NameError: ignored